In [11]:
!pip install gradio ipywidgets


In [12]:
from typing import List, Dict
from dataclasses import dataclass
import json
import random
import gradio as gr


In [13]:
@dataclass
class UserProfile:
    name: str
    age: int
    weight: float
    height: float
    gender: str
    activity_level: str
    goal: str
    restrictions: List[str]
    conditions: List[str]


In [14]:
nutritional_db = {
    "Oatmeal": {"calories": 150, "protein": 5, "carbs": 27, "fat": 3, "tags": ["vegan", "vegetarian"]},
    "Grilled Chicken": {"calories": 250, "protein": 30, "carbs": 0, "fat": 6, "tags": ["keto"]},
    "Quinoa Salad": {"calories": 220, "protein": 8, "carbs": 30, "fat": 7, "tags": ["vegetarian", "vegan"]},
    "Lentils": {"calories": 180, "protein": 12, "carbs": 32, "fat": 0.5, "tags": ["vegan"]},
    "Tofu": {"calories": 200, "protein": 20, "carbs": 5, "fat": 12, "tags": ["vegan"]},
    "Salmon": {"calories": 300, "protein": 34, "carbs": 0, "fat": 20, "tags": []},
    "Boiled Eggs": {"calories": 78, "protein": 6, "carbs": 0.6, "fat": 5, "tags": ["keto"]},
    "Broccoli": {"calories": 55, "protein": 3.7, "carbs": 11, "fat": 0.3, "tags": ["vegan", "vegetarian"]},
    "Almonds": {"calories": 160, "protein": 6, "carbs": 6, "fat": 14, "tags": ["vegan", "vegetarian"]},
    "Greek Yogurt": {"calories": 100, "protein": 10, "carbs": 4, "fat": 5, "tags": ["vegetarian"]}
}


In [15]:
def calculate_bmr(user: UserProfile) -> float:
    if user.gender.lower() == 'male':
        bmr = 10 * user.weight + 6.25 * user.height - 5 * user.age + 5
    else:
        bmr = 10 * user.weight + 6.25 * user.height - 5 * user.age - 161
    return bmr


In [16]:
def calculate_daily_calories(user: UserProfile) -> float:
    activity_multipliers = {
        "sedentary": 1.2,
        "light": 1.375,
        "moderate": 1.55,
        "active": 1.725
    }
    multiplier = activity_multipliers.get(user.activity_level.lower(), 1.2)
    base_calories = calculate_bmr(user) * multiplier
    if user.goal.lower() == "weight loss":
        return base_calories - 500
    elif user.goal.lower() == "muscle gain":
        return base_calories + 500
    else:
        return base_calories


In [17]:
def filter_foods(user: UserProfile) -> List[str]:
    allowed_foods = []
    for food, details in nutritional_db.items():
        if user.restrictions and not all(r.lower() in details['tags'] for r in user.restrictions):
            continue
        if "diabetes" in [c.lower() for c in user.conditions] and nutritional_db[food]["carbs"] > 25:
            continue
        if "heart" in [c.lower() for c in user.conditions] and nutritional_db[food]["fat"] > 15:
            continue
        allowed_foods.append(food)
    return allowed_foods


In [18]:
def generate_meal_plan(user: UserProfile) -> Dict[str, Dict[str, float]]:
    daily_calories = calculate_daily_calories(user)
    allowed_foods = filter_foods(user)
    meal_plan = {}
    total = {"calories": 0.0, "protein": 0.0, "carbs": 0.0, "fat": 0.0}

    attempts = 0
    max_attempts = 100

    while total['calories'] < daily_calories - 100 and allowed_foods and attempts < max_attempts:
        food = random.choice(allowed_foods)
        attempts += 1
        if food in meal_plan:
            continue
        meal_plan[food] = nutritional_db[food]
        for key in total:
            total[key] += float(nutritional_db[food][key])

    return {"meals": meal_plan, "totals": total, "goal": daily_calories}


In [19]:
def create_diet_plan(user_data: dict) -> Dict:
    user = UserProfile(**user_data)
    return generate_meal_plan(user)


In [20]:
def gradio_interface(name, age, weight, height, gender, activity_level, goal, restrictions, conditions):
    user_data = {
        "name": name,
        "age": int(age),
        "weight": float(weight),
        "height": float(height),
        "gender": gender,
        "activity_level": activity_level,
        "goal": goal,
        "restrictions": [r.strip() for r in restrictions.split(',') if r.strip()],
        "conditions": [c.strip() for c in conditions.split(',') if c.strip()]
    }
    result = create_diet_plan(user_data)
    return json.dumps(result, indent=2)

demo = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Name"),
        gr.Number(label="Age"),
        gr.Number(label="Weight (kg)"),
        gr.Number(label="Height (cm)"),
        gr.Radio(["Male", "Female"], label="Gender"),
        gr.Dropdown(["Sedentary", "Light", "Moderate", "Active"], label="Activity Level"),
        gr.Dropdown(["Weight Loss", "Maintain", "Muscle Gain"], label="Fitness Goal"),
        gr.Textbox(label="Dietary Restrictions (comma-separated)"),
        gr.Textbox(label="Health Conditions (comma-separated)")
    ],
    outputs=gr.Textbox(label="Personalized Diet Plan"),
    title="AI Diet Assistant",
    description="Get a personalized diet plan based on your health and goals."
)

if __name__ == "__main__":
    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://bc51b6afb24bb10a71.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
